# Twitter API test

In [1]:
import os
import requests
from pathlib import Path
import json
import tomllib
from dotenv import load_dotenv

In [2]:
dotenv_file = Path('../.env')
load_dotenv(dotenv_file)

True

In [3]:
TWITTER_API_KEY = os.getenv('TWITTER_API_KEY')
TWITTER_API_SECRET = os.getenv('TWITTER_API_SECRET')
TWITTER_BEARER_TOKEN = os.getenv('TWITTER_BEARER_TOKEN')
TWITTER_ACCESS_TOKEN = os.getenv('TWITTER_ACCESS_TOKEN')
TWITTER_ACCESS_SECRET = os.getenv('TWITTER_ACCESS_SECRET')

## via HTTP request

### with cURL

```bash
curl \
    --request GET 'https://api.twitter.com/2/tweets/search/recent?query=from:twitterdev' \
    --header 'Authorization: Bearer $BEARER_TOKEN'
```

In [5]:
def bearer_oauth(request):
    """
    Callable method required by bearer token auth
    """
    request.headers['Authorization'] = f'Bearer {TWITTER_BEARER_TOKEN}'
    request.headers['User-Agent'] = "v2RecentSearchPython"
    return request

search_url = "https://api.twitter.com/2/tweets/search/recent"
# Optional params: start_time,end_time,since_id,until_id,max_results,next_token,
# expansions,tweet.fields,media.fields,poll.fields,place.fields,user.fields
query_params = {'query': '(from:twitterdev -is:retweet) OR #twitterdev','tweet.fields': 'author_id'}

response = requests.get(
    url=search_url,
    auth=bearer_oauth,
    params=query_params,
)

json_response = response.json()
print(json.dumps(json_response, indent=4, sort_keys=True))

{
    "client_id": "27669589",
    "detail": "When authenticating requests to the Twitter API v2 endpoints, you must use keys and tokens from a Twitter developer App that is attached to a Project. You can create a project via the developer portal.",
    "reason": "client-not-enrolled",
    "registration_url": "https://developer.twitter.com/en/docs/projects/overview",
    "required_enrollment": "Appropriate Level of API Access",
    "title": "Client Forbidden",
    "type": "https://api.twitter.com/2/problems/client-forbidden"
}


### Posting with OAuth1

[Link to sample code](https://github.com/twitterdev/Twitter-API-v2-sample-code/blob/main/Manage-Tweets/create_tweet.py)

### Python Wrappers for Twitter API v2

- [tweepy](https://github.com/tweepy/tweepy)
  - docs for OAuth2
- [python-twitter](https://github.com/sns-sdks/python-twitter)